In [18]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [10]:
train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [11]:
trainloader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True, num_workers=2)

test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(test, batch_size=4,shuffle=False, num_workers=2)

Files already downloaded and verified


In [12]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [268]:
class CIFARNet(nn.Module):
    def __init__(self):
        super(CIFARNet, self).__init__()
        self.conv1 = nn.Conv2d(3,5,5)
        self.conv2 = nn.Conv2d(5,8,5)
        self.conv3 = nn.Conv2d(8,10,5)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(10 * 10 * 10, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(F.relu(self.conv3(x)))
        #print(x.shape)
        x = x.view(-1, 10*10*10)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  
        
        return x

In [269]:
net = CIFARNet()

In [270]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [271]:
trial = torch.randn(1,3, 32, 32)
output = net(trial)

# Training Loop

In [272]:
EPOCHS = 5
for epoch in range(EPOCHS):
    running_loss = 0
    for i, inp in enumerate(trainloader):
        inputs, labels = inp
        optimizer.zero_grad()
    
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%3000 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 3000)
            running_loss = 0.0
            
print('Training Done')

Loss [1, 3000](epoch, minibatch):  2.1604747833013533
Loss [1, 6000](epoch, minibatch):  1.949503056983153
Loss [1, 9000](epoch, minibatch):  1.762503723402818
Loss [1, 12000](epoch, minibatch):  1.6403518207371235
Loss [2, 3000](epoch, minibatch):  1.5757869702875613
Loss [2, 6000](epoch, minibatch):  1.5285509405930837
Loss [2, 9000](epoch, minibatch):  1.4847061225523552
Loss [2, 12000](epoch, minibatch):  1.4622294177313646
Loss [3, 3000](epoch, minibatch):  1.4210118033687273
Loss [3, 6000](epoch, minibatch):  1.371557664476335
Loss [3, 9000](epoch, minibatch):  1.3763904208689928
Loss [3, 12000](epoch, minibatch):  1.3607756764839092
Loss [4, 3000](epoch, minibatch):  1.2827279090937227
Loss [4, 6000](epoch, minibatch):  1.2683821771765749
Loss [4, 9000](epoch, minibatch):  1.296918566490213
Loss [4, 12000](epoch, minibatch):  1.2629853349458426
Loss [5, 3000](epoch, minibatch):  1.184413485289241
Loss [5, 6000](epoch, minibatch):  1.1990126163757717
Loss [5, 9000](epoch, minibat

In [273]:
#save model
PATH = './cifar_net.pth'

torch.save(net.state_dict(), PATH)

# Test Model

In [274]:
net = CIFARNet()
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

Accuracy on 10,000 test images:  56.63 %
